# Initialization

## Import

### Import Modules

In [ ]:
import gc
import time

from tasks.optic_disc_cup.datasets import DrishtiDataset, RimOneDataset, DrishtiSimpleDataset
from tasks.optic_disc_cup.metrics import calc_disc_cup_iou
from tasks.optic_disc_cup.losses import DiscCupLoss
from config.config_type import AllConfig, DataConfig, DataTuneConfig, LearnConfig, WeaselConfig, ProtoSegConfig, \
    LossConfig, OptimizerConfig, SchedulerConfig, GuidedNetsConfig
from data.dataset_loaders import DatasetLoaderParamReduced
from learners.protoseg import ProtoSegLearner
from learners.weasel import WeaselLearner
from learners.guidednets import GuidedNetsLearner
from learners.simple_learner import SimpleLearner
from models.u_net import UNet
from data.types import SimpleDatasetKeywordArgs

from torch import cuda, nn, optim

### Autoreload Import

In [ ]:
%reload_ext autoreload
%autoreload 1
%aimport config.config_type
%aimport models.u_net
%aimport data.types, data.dataset_loaders
%aimport learners.meta_learner, learners.weasel, learners.protoseg, learners.guidednets, learners.simple_learner
%aimport tasks.optic_disc_cup.datasets, tasks.optic_disc_cup.metrics, tasks.optic_disc_cup.losses

In [ ]:
%autoreload now

## All Config

### Short Training

In [ ]:
data_config: DataConfig = {
    'num_classes': 3,
    'num_channels': 3,
    'num_workers': 0,
    'batch_size': 1,
    'resize_to': (256, 256)
}

data_tune_config: DataTuneConfig = {
    'shot_list': [5],
    'sparsity_dict': {
        'point': [10],
        'grid': [25],
        'contour': [1],
        'skeleton': [1],
        'region': [1],
        'point_old': [10],
        'grid_old': [25]
    }
}

learn_config: LearnConfig = {
    'should_resume': False,
    'use_gpu': True,
    'num_epochs': 8,
    'exp_name': '',
    'tune_freq': 4,
    'test_freq': 4
}

loss_config: LossConfig = {
    'type': 'ce',
    'ignored_index': -1
}

optimizer_config: OptimizerConfig = {
    'lr': 1e-3,
    'lr_bias': 2 * 1e-3,
    'weight_decay': 5e-5,
    'weight_decay_bias': 0,
    'betas': (0.9, 0.99)
}

scheduler_config: SchedulerConfig = {
    'step_size': 150,
    'gamma': 0.2
}

weasel_config: WeaselConfig = {
    'use_first_order': False,
    'update_param_step_size': 0.3,
    'tune_epochs': 6,
    'tune_test_freq': 3
}

protoseg_config: ProtoSegConfig = {
    'embedding_size': 4
}

guidednets_config: GuidedNetsConfig = {
    'embedding_size': 32
}

all_config: AllConfig = {
    'data': data_config,
    'data_tune': data_tune_config,
    'learn': learn_config,
    'loss': loss_config,
    'optimizer': optimizer_config,
    'scheduler': scheduler_config,
    'weasel': weasel_config,
    'protoseg': protoseg_config,
    'guidednets': guidednets_config
}

### Long Training

In [ ]:
# data_config: DataConfig = {
#     'num_classes': 3,
#     'num_channels': 3,
#     'num_workers': 0,
#     'batch_size': 1,
#     'resize_to': (256, 256)
# }
# 
# data_tune_config: DataTuneConfig = {
#     'shot_list': [10],
#     'sparsity_dict': {
#         'point': [10],
#         'grid': [25],
#         'contour': [1],
#         'skeleton': [1],
#         'region': [1],
#         'point_old': [10],
#         'grid_old': [25]
#     }
# }
# 
# learn_config: LearnConfig = {
#     'should_resume': False,
#     'use_gpu': True,
#     'num_epochs': 200,
#     'exp_name': '',
#     'tune_freq': 40,
#     'test_freq': 100
# }
# 
# loss_config: LossConfig = {
#     'type': 'ce',
#     'ignored_index': -1
# }
# 
# optimizer_config: OptimizerConfig = {
#     'lr': 1e-3,
#     'lr_bias': 2 * 1e-3,
#     'weight_decay': 5e-5,
#     'weight_decay_bias': 0,
#     'betas': (0.9, 0.99)
# }
# 
# scheduler_config: SchedulerConfig = {
#     'step_size': 150,
#     'gamma': 0.2
# }
# 
# weasel_config: WeaselConfig = {
#     'use_first_order': False,
#     'update_param_step_size': 0.3,
#     'tune_epochs': 40,
#     'tune_test_freq': 8
# }
# 
# protoseg_config: ProtoSegConfig = {
#     'embedding_size': 4
# }
# 
# guidednets_config: GuidedNetsConfig = {
#     'embedding_size': 32
# }
# 
# all_config: AllConfig = {
#     'data': data_config,
#     'data_tune': data_tune_config,
#     'learn': learn_config,
#     'loss': loss_config,
#     'optimizer': optimizer_config,
#     'scheduler': scheduler_config,
#     'weasel': weasel_config,
#     'protoseg': protoseg_config,
#     'guidednets': guidednets_config
# }

# Dataset Exploration

## Additional Import

In [ ]:
# import numpy as np
# from matplotlib import pyplot as plt
# 
# from data.types import SparsityValue
# 
# plt.style.use('dark_background')

## RIM-ONE

### Create Dataset

In [ ]:
rim_one_sparsity_params: dict = {
    'point_dot_size': 5,
    'grid_dot_size': 4,
    'contour_radius_dist': 4,
    'contour_radius_thick': 2,
    'skeleton_radius_thick': 4,
    'region_compactness': 0.5
}

rim_one_data = RimOneDataset(
    mode='train',
    num_classes=3,
    num_shots=5,
    resize_to=(256, 256),
    split_seed=0,
    sparsity_params=rim_one_sparsity_params
)

### Check Sparse Masks

In [ ]:
# sparsity_values: dict[str, SparsityValue] = {
#     'point': (1,15),
#     'grid': (15,50),
#     'contour': (0.2,1),
#     'skeleton': (0.2,1), 
#     'region': (0.2,1),
# }
# image, mask, sparse_masks, image_filename = rim_one_data.get_data_with_sparse_all(1, sparsity_values)
# print(image.shape, image.max(), image.min(), image_filename)
# print(mask.shape, mask.dtype, np.unique(mask))
# 
# n_rows = int(np.ceil(len(sparse_masks) / 2)) + 1 
# _, axs = plt.subplots(n_rows, 2, figsize=(12, n_rows*6))
# axs = axs.flat
# axs[0].imshow(image)
# axs[1].imshow(mask)
# for i, sparsity in enumerate(sparse_masks):
#     axs[i+2].imshow(sparse_masks[sparsity])

### Check Others

In [ ]:
# image_sizes = []
# for image_path, mask_path in rim_one_data.get_all_data_path():
#     image, _ = rim_one_data.read_image_mask(image_path, mask_path)
#     image_sizes.append(image.shape)
# 
# image_sizes = np.array(image_sizes)
# 
# print(np.unique(image_sizes[:,0], return_counts=True))
# print(image_sizes[:,0].min(), image_sizes[:,0].max())
# print(np.unique(image_sizes[:,1], return_counts=True))
# print(image_sizes[:,1].min(), image_sizes[:,1].max())

## DRISHTI

### Create Dataset

In [ ]:
drishti_sparsity_params: dict = {
    'point_dot_size': 4,
    'grid_dot_size': 4,
    'contour_radius_dist': 4,
    'contour_radius_thick': 1,
    'skeleton_radius_thick': 3,
    'region_compactness': 0.5
}

drishti_data = DrishtiDataset(
    mode='train',
    num_classes=3,
    num_shots=5,
    resize_to=(256, 256),
    split_seed=0,
    sparsity_params=drishti_sparsity_params
)

### Check Sparse Masks

In [ ]:
# sparsity_values: dict[str, SparsityValue] = {
#     'point': 10,
#     'grid': 25,
#     'contour': 1,
#     'skeleton': 1,
#     'region': 1,
#     'point_old': 10,
#     'grid_old': 25
# }
# image, mask, sparse_masks, image_filename = drishti_data.get_data_with_sparse_all(1, sparsity_values)
# print(image.shape, image.max(), image.min(), image_filename)
# print(mask.shape, mask.dtype, np.unique(mask))
# 
# n_rows = int(np.ceil(len(sparse_masks) / 2)) + 1
# _, axs = plt.subplots(n_rows, 2, figsize=(12, n_rows*6))
# axs = axs.flat
# axs[0].imshow(image)
# axs[1].imshow(mask, cmap='gray')
# for i, sparsity in enumerate(sparse_masks):
#     axs[i+2].imshow(sparse_masks[sparsity])

### Check Others

In [ ]:
# image_sizes = []
# for image_path, mask_path in drishti_data.get_all_data_path():
#     image, _ = rim_one_data.read_image_mask(image_path, mask_path)
#     image_sizes.append(image.shape)
# 
# image_sizes = np.array(image_sizes)
# 
# print(np.unique(image_sizes[:,0], return_counts=True))
# print(image_sizes[:,0].min(), image_sizes[:,0].max())
# print(np.unique(image_sizes[:,1], return_counts=True))
# print(image_sizes[:,1].min(), image_sizes[:,1].max())

## Dataset Loader Params

In [ ]:
rim_one_meta_loader_params: DatasetLoaderParamReduced = {
    'dataset_class': RimOneDataset,
    'dataset_kwargs': {
        'split_seed': 0,
        'split_test_size': 0.2,
        'num_shots': -1,
        'sparsity_mode': 'random',
        'sparsity_value': 'random',
        'sparsity_params': rim_one_sparsity_params
    }
}

drishti_tune_loader_params: DatasetLoaderParamReduced = {
    'dataset_class': DrishtiDataset,
    'dataset_kwargs': {
        'split_seed': 0,
        'split_test_size': 0.2,
        'sparsity_params': drishti_sparsity_params
    }
}

## Num Workers Check

In [ ]:
# from torch.utils.data import DataLoader, Dataset
# 
# def check_num_workers(dataset_instance: Dataset, dataset_name: str, n_workers: int):
#     data_loader = DataLoader(dataset_instance,
#                              batch_size=3,
#                              num_workers=n_workers,
#                              shuffle=True,
#                              pin_memory=True)
#     start_time = time.time()
#     # noinspection PyUnusedLocal
#     for idx, data in enumerate(data_loader):
#         # print("{} - data {}/{}".format(dataset_name, idx+1, len(data_loader)))
#         pass
#     end_time = time.time()
#     print("{} - {} workers: {} seconds".format(dataset_name, num_workers, end_time - start_time))

In [ ]:
# num_workers_range = range(0, 4, 1)
# num_workers_range = range(0, 40, 3)

In [ ]:
# rim_one_dataset = RimOneDataset(
#     'train',
#     all_config['data']['num_classes'],
#     all_config['data']['resize_to'],
#     **rim_one_meta_loader_params['dataset_kwargs']
# )
# 
# for num_workers in num_workers_range:
#     check_num_workers(rim_one_dataset, 'RO', num_workers)

In [ ]:
# drishti_dataset = DrishtiDataset(
#     'train',
#     all_config['data']['num_classes'],
#     all_config['data']['resize_to'],
#     **drishti_tune_loader_params['dataset_kwargs']
# )
# 
# for num_workers in num_workers_range:
#     check_num_workers(drishti_dataset, 'DR', num_workers)

# Weasel Learner

## Initialization

### Update Config

In [ ]:
all_config['data']['batch_size'] = 3
# all_config['data']['batch_size'] = 14

# all_config['data']['num_workers'] = 3

# all_config['learn']['should_resume'] = True

all_config['learn']['exp_name'] = 'v3 RO-DR S WS dummy'
# all_config['learn']['exp_name'] = 'v3 RO-DR L WS'

In [ ]:
rim_one_meta_loader_params['dataset_kwargs']['num_shots'] = 20

### Create Model

In [ ]:
net = UNet(all_config['data']['num_channels'], all_config['data']['num_classes'])

print('# of parameters: ' + str(sum(p.numel() for p in net.parameters() if p.requires_grad)))

### Create Optimizer and Scheduler

In [ ]:
adam_optimizer = optim.Adam([
    {'params': [param for name, param in net.named_parameters() if name[-4:] == 'bias'],
     'lr': all_config['optimizer']['lr_bias'],
     'weight_decay': all_config['optimizer']['weight_decay_bias']},
    {'params': [param for name, param in net.named_parameters() if name[-4:] != 'bias'],
     'lr': all_config['optimizer']['lr'],
     'weight_decay': all_config['optimizer']['weight_decay']}
], betas=all_config['optimizer']['betas'])

step_scheduler = optim.lr_scheduler.StepLR(adam_optimizer,
                                         step_size=all_config['scheduler']['step_size'],
                                         gamma=all_config['scheduler']['gamma'])

### Create Learner

In [ ]:
learner = WeaselLearner(net, all_config, [rim_one_meta_loader_params], drishti_tune_loader_params, 
                        calc_metrics=calc_disc_cup_iou, calc_loss=DiscCupLoss('bce'), optimizer=adam_optimizer, scheduler=step_scheduler)

## Learning

In [ ]:
try:
    learner.learn()
except BaseException as e:
    learner.log_error()
    raise e
finally:
    learner.remove_log_handlers()
    del net, adam_optimizer, step_scheduler, learner
    gc.collect()
    cuda.empty_cache()

In [ ]:
time.sleep(60)

# ProtoSeg Learner

## Initialization

### Update Config

In [ ]:
all_config['data']['batch_size'] = 3
# all_config['data']['batch_size'] = 36

# all_config['data']['num_workers'] = 0

# all_config['learn']['should_resume'] = True

all_config['learn']['exp_name'] = 'v3 RO-DR S PS dummy'
# all_config['learn']['exp_name'] = 'v3 RO-DR L PS'

In [ ]:
rim_one_meta_loader_params['dataset_kwargs']['num_shots'] = 20

### Create Model

In [ ]:
net = UNet(all_config['data']['num_channels'], all_config['protoseg']['embedding_size'])

print('# of parameters: ' + str(sum(p.numel() for p in net.parameters() if p.requires_grad)))

### Create Optimizer and Scheduler

In [ ]:
adam_optimizer = optim.Adam([
    {'params': [param for name, param in net.named_parameters() if name[-4:] == 'bias'],
     'lr': all_config['optimizer']['lr_bias'],
     'weight_decay': all_config['optimizer']['weight_decay_bias']},
    {'params': [param for name, param in net.named_parameters() if name[-4:] != 'bias'],
     'lr': all_config['optimizer']['lr'],
     'weight_decay': all_config['optimizer']['weight_decay']}
], betas=all_config['optimizer']['betas'])

step_scheduler = optim.lr_scheduler.StepLR(adam_optimizer,
                                         step_size=all_config['scheduler']['step_size'],
                                         gamma=all_config['scheduler']['gamma'])

### Create Learner

In [ ]:
learner = ProtoSegLearner(net, all_config, [rim_one_meta_loader_params], drishti_tune_loader_params, 
                          calc_metrics=calc_disc_cup_iou, calc_loss=DiscCupLoss('iou_bce'), optimizer=adam_optimizer, scheduler=step_scheduler)

## Learning

In [ ]:
try:
    learner.learn()
except BaseException as e:
    learner.log_error()
    raise e
finally:
    learner.remove_log_handlers()
    del net, adam_optimizer, step_scheduler, learner
    gc.collect()
    cuda.empty_cache()

# GuidedNet Learner

## Initialization

### Update Config

In [ ]:
all_config['data']['batch_size'] = 2
# all_config['data']['batch_size'] = 36

# all_config['data']['num_workers'] = 0

# all_config['learn']['should_resume'] = True

all_config['learn']['exp_name'] = 'v3 RO-DR S GN dummy'
# all_config['learn']['exp_name'] = 'v3 RO-DR L PS'

In [ ]:
rim_one_meta_loader_params['dataset_kwargs']['num_shots'] = 40

### Create Model

In [ ]:
embedding_size = all_config['guidednets']['embedding_size']

net_image = UNet(all_config['data']['num_channels'], embedding_size, prototype=True).cuda()

net_mask = UNet(1, embedding_size, prototype=True).cuda()
# net_mask = None

net_head = nn.Sequential(
    nn.Conv2d(embedding_size * 2, embedding_size, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.Conv2d(embedding_size, all_config['data']['num_classes'], kernel_size=1)
).cuda()
nn.init.ones_(net_head[0].weight)
nn.init.ones_(net_head[-1].weight)

net_merge = nn.AdaptiveAvgPool2d((1, 1)).cuda()

### Create Optimizer and Scheduler

In [ ]:
net_named_parameters = list(net_image.named_parameters()) + list(net_head.named_parameters()) + list(net_merge.named_parameters())
if net_mask is not None:
    net_named_parameters += net_mask.named_parameters()

adam_optimizer = optim.Adam([
    {'params': [param for name, param in net_named_parameters if name[-4:] == 'bias'],
     'lr': all_config['optimizer']['lr_bias'],
     'weight_decay': all_config['optimizer']['weight_decay_bias']},
    {'params': [param for name, param in net_named_parameters if name[-4:] != 'bias'],
     'lr': all_config['optimizer']['lr'],
     'weight_decay': all_config['optimizer']['weight_decay']}
], betas=all_config['optimizer']['betas'])

step_scheduler = optim.lr_scheduler.StepLR(adam_optimizer,
                                         step_size=all_config['scheduler']['step_size'],
                                         gamma=all_config['scheduler']['gamma'])

### Create Learner

In [ ]:
learner = GuidedNetsLearner(net_image, net_mask, net_merge, net_head, all_config, [rim_one_meta_loader_params], drishti_tune_loader_params,
                            calc_metrics=calc_disc_cup_iou, calc_loss=DiscCupLoss('ce'), optimizer=adam_optimizer, scheduler=step_scheduler)

## Learning

In [ ]:
try:
    learner.learn()
except BaseException as e:
    learner.log_error()
    raise e
finally:
    learner.remove_log_handlers()
    del net_image, net_mask, net_head, net_merge, adam_optimizer, step_scheduler, learner
    gc.collect()
    cuda.empty_cache()

# Simple Learner

## Initialization

### Update Config

In [ ]:
all_config['data']['batch_size'] = 8

all_config['learn']['exp_name'] = 'v3 DR S SL'

### Set Dataset Params

In [ ]:
drishti_params: SimpleDatasetKeywordArgs = {
    'split_seed': 0,
    'split_val_size': 0.15,
    'split_test_size': 0.15,
}

### Create Model

In [ ]:
net = UNet(all_config['data']['num_channels'], all_config['data']['num_classes'])

print('# of parameters: ' + str(sum(p.numel() for p in net.parameters() if p.requires_grad)))

### Create Optimizer and Scheduler

In [ ]:
adam_optimizer = optim.Adam([
    {'params': [param for name, param in net.named_parameters() if name[-4:] == 'bias'],
     'lr': all_config['optimizer']['lr_bias'],
     'weight_decay': all_config['optimizer']['weight_decay_bias']},
    {'params': [param for name, param in net.named_parameters() if name[-4:] != 'bias'],
     'lr': all_config['optimizer']['lr'],
     'weight_decay': all_config['optimizer']['weight_decay']}
], betas=all_config['optimizer']['betas'])

step_scheduler = optim.lr_scheduler.StepLR(adam_optimizer,
                                         step_size=all_config['scheduler']['step_size'],
                                         gamma=all_config['scheduler']['gamma'])

### Create Learner

In [ ]:
learner = SimpleLearner(net, all_config, DrishtiSimpleDataset, drishti_params, 
                        calc_metrics=calc_disc_cup_iou, calc_loss=DiscCupLoss('ce'), optimizer=adam_optimizer, scheduler=step_scheduler)

## Learning

In [ ]:
try:
    learner.learn()
except BaseException as e:
    learner.log_error()
    raise e
finally:
    learner.remove_log_handlers()
    del net, adam_optimizer, step_scheduler, learner
    gc.collect()
    cuda.empty_cache()

# Other